In [5]:
from tokenizers import ByteLevelBPETokenizer
from transformers import BartConfig, BartTokenizerFast, AutoModelForMaskedLM
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import pipeline, Trainer, TrainingArguments
import pandas as pd

In [2]:
tokenizer = ByteLevelBPETokenizer()
paths = ["/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/europarl-v7.da-en.da"]
tokenizer.train(files=paths, vocab_size=50000, min_frequency=2, special_tokens=[
    "<s>",
    "</s>",
    "<pad>"
])

In [3]:
tokenizer.save_model("BartTokenizer")

['BartTokenizer/vocab.json', 'BartTokenizer/merges.txt']

### Load data

In [8]:
data = pd.read_csv("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/SentToSent.csv", sep=";")
lines_without_comma = data["withoutComma"]
lines_with_comma = data["withComma"]
lines_without_comma[4], lines_with_comma[4]

('i mellemtiden ønsker jeg som også en del kolleger har anmodet om at vi iagttager et minuts stilhed til minde om ofrene for bla stormene i de medlemslande der blev ramt',
 'i mellemtiden ønsker jeg, som også en del kolleger har anmodet om, at vi iagttager et minuts stilhed til minde om ofrene for bla stormene i de medlemslande, der blev ramt')

### Test tokenizer

In [9]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./BartTokenizer/vocab.json",
    "./BartTokenizer/merges.txt",)

tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)

tokenizer.enable_truncation(max_length=512)

# print(torch.cuda.is_available())  GPU P100

print(tokenizer.encode('Et tu').tokens)

['<s>', 'Et', 'Ġt', 'u', '</s>']


### Initalize BART model

In [16]:
tokenizer = BartTokenizerFast.from_pretrained("./BartTokenizer", max_len=2000)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
config = BartConfig(
    vocab_size=50000,
    max_position_embeddings=64,
    num_attention_heads=4,
    num_hidden_layers=4,
    type_vocab_size=1,
)

model = AutoModelForMaskedLM.from_config(config=config)
print(model.num_parameters())

404054016
